# N5GEH-Sev datamodeling workshop
A data model is a conceptual representation of data objects, associations, and constraints. It provides a structured framework for understanding and organizing information within a database system, e.g. FIWARE stack.

This example shows a workflow, how you can define or reuse use case specific data models and ensure FIWARE compatibility by merging these models with existing data model in FiLiP. The merged models can be used for interaction with FIWARE platform and in other information processing systems to establish interoperability.

## Use Case
In this example, our use case is to capture data and information of a weather station in FIWARE stack. We will first create data models for necessary components. And then, we will also showcase how to export, share and reuse the data models.

## Import dependency

In [1]:
from pydantic import ConfigDict, BaseModel
from pydantic.fields import Field, FieldInfo
from typing import Any, Optional
from filip.models import FiwareHeader
from filip.models.ngsi_v2.context import ContextEntityKeyValues
from filip.clients.ngsi_v2.cb import ContextBrokerClient

## Create data model
Good data modelling can harmonise representation formats and semantics for various applications. Therefore, we try to reuse existing data models. For our specific use case, we can also define our own data models.

### Common data models
Some data models are use case agnostic. For example, we reuse the data model of `PostalAddress` from [https://schema.org/](https://schema.org/) to represent the address of the weather station

In [2]:
class PostalAddress(BaseModel):
    """
    https://schema.org/PostalAddress
    """

    model_config = ConfigDict(populate_by_name=True, coerce_numbers_to_str=True)  # Pydantic specific settings

    addressCountry: str = Field(
        alias="addressCountry",
        description="County code according to ISO 3166-1-alpha-2",
    )
    streetAddress: str = Field(
        alias="streetAddress",
        description="The street address. For example, 1600 Amphitheatre Pkwy.",
    )
    addressRegion: Optional[str] = Field(
        alias="addressRegion",
        default=None,
    )
    addressLocality: Optional[str] = Field(
        alias="addressLocality",
        default=None,
        description="The locality in which the street address is, and which is "
                    "in the region. For example, Mountain View.",
    )
    postalCode: Optional[str] = Field(
        alias="postalCode",
        default=None,
        description="The postal code. For example, 94043.",
    )

### Domain data models
Some other data models are related with a specific domain. For example, [Brick](https://brickschema.org/) provides an open-source standardized semantic description in building domain.

In [3]:
class OutsideAirTemperatureSensor(BaseModel):
    temperature: float = Field(
        default=20.0,
        description="brick:Quantity, measure temperature in degree Celsius"
    )

class OutsideAirHumiditySensor(BaseModel):
    humidity: float = Field(
        default=50.0,
        description="brick:Quantity, measure humidity in percentage"
    )

class WindDirectionSensor(BaseModel):
    windDirection: float = Field(
        default=90.0,
        description="brick:Quantity, measure wind direction in degrees (0 degree for North)"
    )

class WindSpeedSensor(BaseModel):
    windSpeed: float = Field(
        default=50.0,
        description="brick:Quantity, measure wind speed in m/s"
    )

class WeatherStation(BaseModel):
    model_config = ConfigDict(coerce_numbers_to_str=True, extra="ignore")  # Pydantic specific settings
    # TODO Any
    hasOutsideAirTemperatureSensor: Any = Field(
        default=None,
        description="brick:hasPoint, link with a URI of OutsideAirTemperatureSensor",
    )
    hasOutsideAirHumiditySensor: Any = Field(
        default=None,
        description="brick:hasPoint, link with a URI of OutsideAirHumiditySensor",
    )
    hasWindDirectionSensor: Any = Field(
        default=None,
        description="brick:hasPoint, link with a URI of WindDirectionSensor",
    )
    hasWindSpeedSensor: Any = Field(
        default=None,
        description="brick:hasPoint, link with a URI of WindSpeedSensor",
    )
    hasAddress: PostalAddress

## Compliance with FIWARE
After creating the data models, we also want to ensure the FIWARE compatibility. For this purpose, we can simply merge the data models of our use cases with the existing FIWARE models in `FiLiP`

In [4]:
# Merge models with simplified entity model
class WeatherStationFIWARE(WeatherStation, ContextEntityKeyValues):
    # add default for type if not explicitly set
    type: str = FieldInfo.merge_field_infos(
        # Field info of the general FIWARE data model in FiLiP
        ContextEntityKeyValues.model_fields["type"],
        # set the default value
        default="WeatherStation",
        description="Type of the Weather Station",
    )
class OutsideAirTemperatureSensorFIWARE(OutsideAirTemperatureSensor, ContextEntityKeyValues):
    type: str = FieldInfo.merge_field_infos(
        # Field info of the general FIWARE data model in FiLiP
        ContextEntityKeyValues.model_fields["type"],
        default="OutsideAirTemperatureSensor"
    )
class OutsideAirHumiditySensorFIWARE(OutsideAirHumiditySensor, ContextEntityKeyValues):
    type: str = FieldInfo.merge_field_infos(
        # Field info of the general FIWARE data model in FiLiP
        ContextEntityKeyValues.model_fields["type"],
        default="OutsideAirHumiditySensor"
    )

class WindDirectionSensorFIWARE(WindDirectionSensor, ContextEntityKeyValues):
    type: str = FieldInfo.merge_field_infos(
        # Field info of the general FIWARE data model in FiLiP
        ContextEntityKeyValues.model_fields["type"],
        default="WindDirectionSensor"
    )

class WindSpeedSensorFIWARE(WindSpeedSensor, ContextEntityKeyValues):
    type: str = FieldInfo.merge_field_infos(
        # Field info of the general FIWARE data model in FiLiP
        ContextEntityKeyValues.model_fields["type"],
        default="WindSpeedSensor"
    )

## Using data models for internal applications
Initializing FIWARE API client

In [5]:
CB_URL = "http://134.130.166.184:1026"
SERVICE = 'filip'  # FIWARE-Service
SERVICE_PATH = '/'  # FIWARE-Servicepath
fiware_header = FiwareHeader(service=SERVICE,
                             service_path=SERVICE_PATH)
cb_client = ContextBrokerClient(url=CB_URL, fiware_header=fiware_header)

Provision data points for sensors using data models

In [6]:
t_sen = OutsideAirTemperatureSensorFIWARE(id="OutsideAirTemperatureSensor:01")
h_sen = OutsideAirHumiditySensorFIWARE(id="OutsideAirHumiditySensor:01")
wind_d = WindDirectionSensorFIWARE(id="WindDirectionSensor:01")
wind_s = WindSpeedSensorFIWARE(id="WindSpeedSensor:01")
for entity in (t_sen, h_sen, wind_s, wind_d):
    cb_client.post_entity(entity=entity, key_values=True, update=True)

Create weather station and link it with sensors using data models

In [7]:
ws = WeatherStationFIWARE(
    id="WeatherStation:01",
    hasAddress={
            "addressCountry": "Germany",
            "streetAddress": "Mathieustr. 10",
            "postalCode": 52072,
        },
)
ws.hasOutsideAirTemperatureSensor = t_sen.id
ws.hasOutsideAirHumiditySensor = h_sen.id
ws.hasWindDirectionSensor = wind_d.id
ws.hasWindSpeedSensor = wind_s.id
cb_client.post_entity(entity=ws, key_values=True, update=True)

Read and use data based on data model

In [8]:
# Read weather station
ws_data = cb_client.get_entity(entity_id="WeatherStation:01", response_format="keyValues").model_dump()
print(ws_data)
ws_general = WeatherStation.model_validate(ws_data)
print(ws_general.model_dump_json(indent=2))

address = ws_general.hasAddress
print(address.model_dump_json(indent=2))

# Read temperature
temperature_sensor = OutsideAirTemperatureSensor.model_validate(
    cb_client.get_entity(
    entity_id=ws_general.hasOutsideAirTemperatureSensor,
    response_format="keyValues").model_dump()
)
print(f"Current temperature: {temperature_sensor.temperature}")

{'id': 'WeatherStation:01', 'type': 'WeatherStation', 'hasAddress': {'addressCountry': 'Germany', 'streetAddress': 'Mathieustr. 10', 'addressRegion': None, 'addressLocality': None, 'postalCode': '52072'}, 'hasOutsideAirHumiditySensor': 'OutsideAirHumiditySensor:01', 'hasOutsideAirTemperatureSensor': 'OutsideAirTemperatureSensor:01', 'hasWindDirectionSensor': 'WindDirectionSensor:01', 'hasWindSpeedSensor': 'WindSpeedSensor:01'}
{
  "hasOutsideAirTemperatureSensor": "OutsideAirTemperatureSensor:01",
  "hasOutsideAirHumiditySensor": "OutsideAirHumiditySensor:01",
  "hasWindDirectionSensor": "WindDirectionSensor:01",
  "hasWindSpeedSensor": "WindSpeedSensor:01",
  "hasAddress": {
    "addressCountry": "Germany",
    "streetAddress": "Mathieustr. 10",
    "addressRegion": null,
    "addressLocality": null,
    "postalCode": "52072"
  }
}
{
  "addressCountry": "Germany",
  "streetAddress": "Mathieustr. 10",
  "addressRegion": null,
  "addressLocality": null,
  "postalCode": "52072"
}
Current

## Using data models for external applications
So far, data models are solely used by "us," who created them. But what if we have to share the data with partners or external applications? In such scenarios, data models remain highly valuable, facilitating seamless data exchange through validation mechanisms and even programmable objects.

### Export serialized data models
In order to share data models, we usually need to export the Pydantic classes to serialized data format, i.e. JSON Schema in this example.

In [9]:
from pprint import pprint
import json

weather_station_schema = WeatherStation.model_json_schema()
with open("./weather_station_schema.json", "w") as f:
    json.dump(weather_station_schema, f)
pprint(weather_station_schema)

weather_station_fiware_schema = WeatherStationFIWARE.model_json_schema()
with open("./weather_station_fiware_schema.json", "w") as f:
    json.dump(weather_station_fiware_schema, f)
pprint(weather_station_fiware_schema)

{'$defs': {'PostalAddress': {'description': 'https://schema.org/PostalAddress',
                             'properties': {'addressCountry': {'description': 'County '
                                                                              'code '
                                                                              'according '
                                                                              'to '
                                                                              'ISO '
                                                                              '3166-1-alpha-2',
                                                               'title': 'Addresscountry',
                                                               'type': 'string'},
                                            'addressLocality': {'anyOf': [{'type': 'string'},
                                                                          {'type': 'null'}],
                               

### Use data models in other applications
As you can see above, the exported data model is based on normal JSON syntax, and is therefore both machine and human readable. There are various libraries, that support parsing this data. For example, in Python `jsonschema` can be use to validate the data against the data model, and `datamodel-code-generator` can be used to generate python objects.

In [10]:
# Validation
from jsonschema import validate
ws_data = cb_client.get_entity(entity_id="WeatherStation:01", response_format="keyValues").model_dump()
validate(instance=ws_data, schema=weather_station_schema)
print("The data conforms to the data model")

The data conforms to the data model


In [11]:
# Parse serialized models
from pathlib import Path
from datamodel_code_generator import generate, DataModelType, InputFileType
output_path=Path("./output/weather_station.py")
class_name="GeneratedWeatherStation"
generate(Path("./weather_station_schema.json"),
         class_name=class_name,
         output=output_path,
         output_model_type=DataModelType.PydanticV2BaseModel,
         use_subclass_enum=True,
         input_file_type=InputFileType.JsonSchema
        )
print(f"Pyhton object '{class_name}' generated in '{output_path}'")

Pyhton object 'GeneratedWeatherStation' generated in 'output\weather_station.py'


In [12]:
# Use generated python object
from output.weather_station import GeneratedWeatherStation
ws_data = cb_client.get_entity(entity_id="WeatherStation:01", response_format="keyValues").model_dump()
ws_generated = GeneratedWeatherStation.model_validate(ws_data)
print(ws_generated.model_dump_json(indent=2))
print(ws_generated.hasAddress.model_dump_json(indent=2))

{
  "hasOutsideAirTemperatureSensor": "OutsideAirTemperatureSensor:01",
  "hasOutsideAirHumiditySensor": "OutsideAirHumiditySensor:01",
  "hasWindDirectionSensor": "WindDirectionSensor:01",
  "hasWindSpeedSensor": "WindSpeedSensor:01",
  "hasAddress": {
    "addressCountry": "Germany",
    "streetAddress": "Mathieustr. 10",
    "addressRegion": null,
    "addressLocality": null,
    "postalCode": "52072"
  }
}
{
  "addressCountry": "Germany",
  "streetAddress": "Mathieustr. 10",
  "addressRegion": null,
  "addressLocality": null,
  "postalCode": "52072"
}
